In [3]:
import pandas as pd
import polars as pl

In [8]:
FILE_PATH = 'MycroftMind_challenge_dataset.xlsx'

In [49]:
def print_sheet_names(file_path):
    data_file = pd.ExcelFile(file_path)
    names = list(data_file.sheet_names)[1:]
    print(f'Sheet names: {names}')
    for name in names:
        sheet_data = pd.read_excel(file_path, name)
        print(f'\t{name}\'s headers: {sheet_data.columns.to_list()}')

def merge_data(file_path):
    data_file = pd.ExcelFile(file_path)
    sheet_names = list(data_file.sheet_names[1:])
    merged_data = pd.read_excel(file_path, sheet_names[0]) #Imported and exported Energy
    next_data = pd.read_excel(file_path, sheet_names[1]) #Predicted energy
    merged_data = pd.merge(merged_data, next_data, on=['DeviceID', 'Timestamp'])
    next_data = pd.read_excel(file_path, sheet_names[2]).drop(columns=['Imported_energy (kWh)']) #Flexibility
    merged_data = pd.merge(merged_data, next_data, on=['DeviceID', 'Timestamp'])
    next_data = pd.read_excel(file_path, sheet_names[3]) #Battery
    merged_data = pd.merge(merged_data, next_data, on=['DeviceID', 'Timestamp'])
    next_data = pd.read_excel(file_path, sheet_names[4]) #Battery params 
    merged_data = pd.merge(merged_data, next_data, on=['DeviceID'], how='left')
    next_data = pd.read_excel(file_path, sheet_names[5]) #Weather
    merged_data = pd.merge(merged_data, next_data, on=['Timestamp'], how='left')
    next_data = pd.read_excel(file_path, sheet_names[6]) #Market prices
    merged_data = pd.merge(merged_data, next_data, on=['Timestamp'], how='left')
    return merged_data

def transform_columns(df: pl.DataFrame):
    return (
        df
        .with_columns(
            Timestamp=pl.col('Timestamp').str.to_datetime(),
            Local_demand=pl.col('Battery_capacity (kWh)') - pl.col('Stored_energy (kWh)'),
            Spot_Price_EUR=pl.col('Spot_Price (EUR/MWh)').str.replace(',', '.').cast(float),
            Spot_Price_CZK=pl.col('Spot_Price (CZK/MWh)').str.replace(',', '.').cast(float),
        )
        .drop(['Spot_Price (EUR/MWh)', 'Spot_Price (CZK/MWh)'])
    )

In [50]:
df_pandas = merge_data(FILE_PATH)

df = transform_columns(pl.from_pandas(df_pandas))

df.head(10)

DeviceID,Timestamp,Imported_energy (kWh),Exported_energy (kWh),Predicted_Imported_energy (kWh),Predicted_Exported_energy (kWh),Flexible_consumption (kWh),Flexibility_demand_amount [-1;1],Flexibility_demand_price (CZK/MWh),Stored_energy (kWh),Battery_capacity (kWh),Max_charge_rate (kW),Init_capacity (%),t_2m:C,t_apparent:C,relative_humidity_2m:p,precip_15min:mm,diffuse_rad:W,direct_rad:W,global_rad:W,wind_speed_10m:ms,wind_dir_10m:d,uv:idx,Local_demand,Spot_Price_EUR,Spot_Price_CZK
str,datetime[μs],f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""OM1""",2024-01-01 00:00:00,0.044,0.0,0.038339,0.0,0.0,-0.456757,921.550783,7.0,14,6,50,2.4,2.4,99.3,0.0,0.0,0.0,0.0,0.7,9.8,0.0,7.0,0.1,2.47
"""OM1""",2024-01-01 00:05:00,0.04,0.0,0.036386,0.0,0.0,-0.617518,855.522083,6.956,14,6,50,null,null,null,null,null,null,null,null,null,null,7.044,null,null
"""OM1""",2024-01-01 00:10:00,0.04,0.0,0.037537,0.0,0.0,-0.915563,1007.028267,6.916,14,6,50,null,null,null,null,null,null,null,null,null,null,7.084,null,null
"""OM1""",2024-01-01 00:15:00,0.038,0.0,0.045759,0.0,0.0,-0.506675,883.924399,6.876,14,6,50,2.5,2.5,97.6,0.03,0.0,0.0,0.0,1.1,48.9,0.0,7.124,null,null
"""OM1""",2024-01-01 00:20:00,0.056,0.0,0.057124,0.0,0.0,-0.460941,576.059459,6.838,14,6,50,null,null,null,null,null,null,null,null,null,null,7.162,null,null
"""OM1""",2024-01-01 00:25:00,0.069,0.0,0.066167,0.0,0.0,-0.584217,940.633876,6.782,14,6,50,null,null,null,null,null,null,null,null,null,null,7.218,null,null
"""OM1""",2024-01-01 00:30:00,0.071,0.0,0.066228,0.0,0.0,-0.565535,925.496707,6.713,14,6,50,2.2,2.2,98.7,0.02,0.0,0.0,0.0,1.1,118.0,0.0,7.287,null,null
"""OM1""",2024-01-01 00:35:00,0.074,0.0,0.080358,0.0,0.0,-0.49456,767.193527,6.642,14,6,50,null,null,null,null,null,null,null,null,null,null,7.358,null,null
"""OM1""",2024-01-01 00:40:00,0.075,0.0,0.078008,0.0,0.0,-0.692883,824.589737,6.568,14,6,50,null,null,null,null,null,null,null,null,null,null,7.432,null,null
